# Debug Drill: The Hallucinating RAG

**Scenario:**
You built a RAG system to answer questions about your company's policies.

"The bot told me we have a 90-day return policy!" a customer complains.

But your actual policy is 30 days. The LLM is hallucinating despite having context.

**Your Task:**
1. Identify why the RAG system is hallucinating
2. Implement faithfulness checking
3. Fix the prompt to reduce hallucinations
4. Write a 3-bullet postmortem

---

In [ ]:
import re

# Knowledge base (ground truth)
KNOWLEDGE_BASE = {
    "return_policy": "We accept returns within 30 days of purchase. Items must be unused and in original packaging.",
    "shipping": "Standard shipping takes 5-7 business days. Express shipping delivers in 2-3 days for $9.99.",
    "warranty": "All products come with a 1-year limited warranty covering manufacturing defects.",
    "refunds": "Refunds are processed within 5-10 business days after we receive the returned item.",
}

In [ ]:
# ===== COLLEAGUE'S CODE (BUG: Weak prompt allows hallucination) =====

def colleague_rag_prompt(query, context):
    """Weak prompt that allows hallucination."""
    # BUG: Prompt doesn't strongly constrain to context
    return f"""Answer this question: {query}

Here's some information that might help:
{context}

Answer:"""

def simulate_llm_response(prompt, hallucination_prone=True):
    """
    Simulate LLM responses.
    When hallucination_prone=True, the model may add incorrect information.
    """
    # Extract context from prompt
    context_match = re.search(r"might help:\n(.+?)\n\nAnswer", prompt, re.DOTALL)
    context = context_match.group(1) if context_match else ""
    
    # Simulated responses (some hallucinate, some don't)
    if "return" in prompt.lower():
        if hallucination_prone:
            # HALLUCINATION: Says 90 days instead of 30!
            return "Our return policy allows returns within 90 days of purchase. Items should be in good condition."
        else:
            return "Based on the policy, returns are accepted within 30 days. Items must be unused and in original packaging."
    
    if "shipping" in prompt.lower():
        if hallucination_prone:
            # HALLUCINATION: Free shipping not mentioned in context
            return "Shipping takes 5-7 days for standard, 2-3 days for express. Free shipping on orders over $50!"
        else:
            return "Standard shipping is 5-7 business days. Express shipping is 2-3 days for $9.99."
    
    return "I don't have information about that."

# Test the broken system
query = "What is your return policy?"
context = KNOWLEDGE_BASE["return_policy"]
prompt = colleague_rag_prompt(query, context)
response = simulate_llm_response(prompt, hallucination_prone=True)

print("=== Colleague's RAG System ===")
print(f"\nQuery: {query}")
print(f"\nContext (TRUTH): {context}")
print(f"\nResponse: {response}")
print(f"\n❌ HALLUCINATION: Response says '90 days' but context says '30 days'!")

---

## Your Investigation

### Step 1: Implement faithfulness checking

In [ ]:
def check_faithfulness(response, context):
    """
    Check if response contains claims not supported by context.
    Returns (is_faithful, issues_found)
    """
    issues = []
    
    # Extract numbers from both
    response_numbers = set(re.findall(r'\d+', response))
    context_numbers = set(re.findall(r'\d+', context))
    
    # Numbers in response not in context = potential hallucination
    hallucinated_numbers = response_numbers - context_numbers
    if hallucinated_numbers:
        issues.append(f"Numbers not in context: {hallucinated_numbers}")
    
    # Check for common hallucination patterns
    hallucination_phrases = [
        ("free shipping", "free_shipping"),
        ("lifetime", "lifetime_warranty"),
        ("unlimited", "unlimited_claim"),
    ]
    
    for phrase, key in hallucination_phrases:
        if phrase in response.lower() and phrase not in context.lower():
            issues.append(f"Claim not in context: '{phrase}'")
    
    is_faithful = len(issues) == 0
    return is_faithful, issues

# Test faithfulness checker
is_faithful, issues = check_faithfulness(response, context)

print("=== Faithfulness Check ===")
print(f"Response: {response[:80]}...")
print(f"\nFaithful: {is_faithful}")
if issues:
    print("Issues found:")
    for issue in issues:
        print(f"  ❌ {issue}")

### Step 2: TODO - Fix the prompt

In [ ]:
# TODO: Create a better prompt that reduces hallucination

# Uncomment and complete:

# def fixed_rag_prompt(query, context):
#     """Strong prompt that constrains to context."""
#     return f"""You are a helpful assistant. Answer the question using ONLY the information provided below.
# 
# IMPORTANT RULES:
# 1. ONLY use facts from the CONTEXT below
# 2. Do NOT add information not in the context
# 3. If the answer is not in the context, say "I don't have that specific information"
# 4. Quote specific numbers and policies exactly as they appear
# 
# CONTEXT:
# {context}
# 
# QUESTION: {query}
# 
# ANSWER (using only the context above):"""
# 
# # Test with fixed prompt
# fixed_prompt = fixed_rag_prompt(query, context)
# fixed_response = simulate_llm_response(fixed_prompt, hallucination_prone=False)
# 
# print("=== Fixed RAG Prompt ===")
# print(fixed_prompt[:300] + "...")
# print(f"\nResponse: {fixed_response}")

In [ ]:
# TODO: Verify the fixed response is faithful

# Uncomment:

# is_faithful_fixed, issues_fixed = check_faithfulness(fixed_response, context)
# 
# print("=== Comparison ===")
# print(f"\nOriginal Response:")
# print(f"  Faithful: {is_faithful}")
# print(f"  Issues: {issues}")
# 
# print(f"\nFixed Response:")
# print(f"  Faithful: {is_faithful_fixed}")
# print(f"  Issues: {issues_fixed}")

In [ ]:
# TODO: Test on multiple queries

# Uncomment:

# test_cases = [
#     ("What is your return policy?", "return_policy"),
#     ("How long does shipping take?", "shipping"),
#     ("What warranty do you offer?", "warranty"),
# ]
# 
# print("=== Full System Test ===")
# for query, context_key in test_cases:
#     context = KNOWLEDGE_BASE[context_key]
#     prompt = fixed_rag_prompt(query, context)
#     response = simulate_llm_response(prompt, hallucination_prone=False)
#     is_faithful, issues = check_faithfulness(response, context)
#     
#     status = "✓" if is_faithful else "❌"
#     print(f"\n{status} Query: {query}")
#     print(f"   Response: {response[:60]}...")
#     if issues:
#         print(f"   Issues: {issues}")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert callable(fixed_rag_prompt), "Should have created fixed_rag_prompt function"
# assert is_faithful_fixed, "Fixed response should be faithful to context"
# assert "30" in fixed_response, "Fixed response should mention 30 days (from context)"
# 
# print("✓ RAG hallucination fixed!")
# print("✓ Prompt now constrains to context")
# print("✓ Faithfulness checking implemented")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Hallucinating RAG

### What happened:
- (Your answer: What incorrect information did the system provide?)

### Root cause:
- (Your answer: Why did the LLM hallucinate despite having correct context?)

### How to prevent:
- (Your answer: What prompt techniques and checks reduce hallucination?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **LLMs can hallucinate even with context.** The prompt must strongly constrain.

2. **"ONLY use the context" is key.** Explicitly forbid outside knowledge.

3. **Faithfulness checking catches hallucinations.** Check numbers, claims, and dates.

4. **"I don't know" is a valid answer.** Better than making things up.

---

## Anti-Hallucination Techniques

| Technique | How It Helps |
|-----------|-------------|
| "ONLY use context" | Constrains to provided info |
| "Say I don't know" | Allows safe fallback |
| Quote exact numbers | Reduces numeric drift |
| Faithfulness check | Catches issues post-hoc |
| Citation requirement | Forces grounding |